# Landslide Hazard Analysis Using SAR
Intro to the use case/methods used

Search for SLC images over Haiti in a defined time period

In [ ]:
from datetime import datetime

start_time = datetime.strptime('2017-08-01T23:59', '%Y-%m-%dT%H:%M')
end_time = datetime.strptime('2021-08-16T23:59', '%Y-%m-%dT%H:%M')
event_time = datetime.strptime('2021-08-14T00:00', '%Y-%m-%dT%H:%M')

import asf_search
scenes_to_submit = []
wkt = 'POLYGON((-74.6 18.0,-73.0 18.0,-73.0 18.8,-74.6 18.8, -74.6 18.0))'
results = asf_search.geo_search(platform=[asf_search.PLATFORM.SENTINEL1], intersectsWith=wkt, processingLevel='SLC', start=start_time, end=end_time)
[scenes_to_submit.append(result.properties['sceneName']) for result in results]
print(f'There are {len(scenes_to_submit)} scenes in the AOI between {start_time} and {end_time}')

Request HyP3 processing of Haiti data for two years

In [ ]:
import hyp3_sdk as sdk

print(f'Submitting {len(scenes_to_submit)} jobs')
hyp3 = sdk.HyP3()
rtc_jobs = sdk.Batch()
for scene in scenes_to_submit:
    rtc_jobs += hyp3.submit_rtc_job(granule=scene, name='IGARSS-HAITI')

In [ ]:
rtc_jobs = hyp3.watch(rtc_jobs)
succeeded_jobs = rtc_jobs.filter_jobs(succeeded=True, running=False, failed=False)
file_list = succeeded_jobs.download_files()

Download the staged version of the data

In [ ]:
import boto3
import os
import zipfile
from pathlib import Path
from botocore import UNSIGNED
from botocore.client import Config

s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))

for file in file_list:
    download_path = Path(file)
    s3_client.download_file('jrsmale-shenanigans', f'igarss_2023/{file}', download_path)
    with zipfile.ZipFile(download_path, 'r') as zip_ref:
        zip_ref.extractall(download_path.stem)
    os.unlink(download_path)

Crop all images to same extent (e.g. InSAR Notebook)

In [ ]:
from pathlib import Path
from typing import List, Union
from osgeo import gdal

def clip_hyp3_products_to_same_extent(data_dir: Union[str, Path], extent: List[float]) -> List[str]:
    """Clip all GeoTIFF hyp3 products to a specified extent
    Args:
        data_dir:
            directory containing the GeoTIFF files to clip
        extent:
            a list of the upper-left x, upper-left y, lower-right x, and lower-right y corner coordinates of desired extent.
    Returns: None
    """
    files_for_xarray = ['VV.tif', 'VH.tif', '_ls_map.tif']

    for extension in files_for_xarray:
        print(extension)
        for file in data_dir.glob(f'**/*{extension}'):
            dst_file = Path(str(file.absolute()).replace(f'{extension}', f'{extension[:-4]}_clipped.tif'))
            if not Path(dst_file).is_file():
                gdal.Translate(destName=str(dst_file), srcDS=str(file), projWin=extent)#, projWinSRS="EPSG:4326")

In [ ]:
data_dir = Path('/Users/jrsmale/data/unzipped_IGARSS/')
extent = [544823, 2071376, 683411, 1987136] #TODO: same/similar coords different proj because gdal didn't like EPSG:4326
clip_hyp3_products_to_same_extent(data_dir, extent)

Load the geotiffs into Xarray with datetime stamps for each image

In [ ]:
def extract_tif_names(scene_dir_path):
    scene_path_listed = os.listdir(scene_dir_path)
    scene_files_vv = [fname for fname in scene_path_listed if fname.endswith('_VV_clipped.tif')]
    scene_files_vh = [fname for fname in scene_path_listed if fname.endswith('_VH_clipped.tif')]
    scene_files_ls = [fname for fname in scene_path_listed if fname.endswith('_ls_map_clipped.tif')]
    scene_file_rm = [fname for fname in scene_path_listed if fname.endswith('README.md.txt')]
    return scene_files_vv, scene_files_vh, scene_files_ls, scene_file_rm

scenes_listed = os.listdir(data_dir)
fpaths_vv, fpaths_vh, fpaths_ls, fpaths_rm = [],[],[], []

for element in range(len(scenes_listed)):
    print(f'{data_dir}/{scenes_listed[element]}')
    try:
        good_files = extract_tif_names(f'{data_dir}/{scenes_listed[element]}')
        path_vh = f'{data_dir}/{scenes_listed[element]}/{good_files[0][0]}'
        path_vv = f'{data_dir}/{scenes_listed[element]}/{good_files[1][0]}'
        path_ls = f'{data_dir}/{scenes_listed[element]}/{good_files[2][0]}'
        path_readme = f'{data_dir}/{scenes_listed[element]}/{good_files[3][0]}'

        fpaths_vv.append(path_vv)
        fpaths_vh.append(path_vh)
        fpaths_ls.append(path_ls)
        fpaths_rm.append(path_readme)
    except:
        print(f'Couldnt use {scenes_listed[element]}. Skipping.')
        continue

Pre-temporal averaging, create time series plot of single pixel (or group of pixels)

In [ ]:
import xarray as xr

def preprocess(da_orig, file_type):
    '''function that should return an xarray object with time dimension and associated metadata given a path to a single RTC scene, if its dualpol will have multiple bands, currently just as 2 data arrays but could merge.
    goal would be to apply this a list of directories for different RTC products, return cube along time dimension - I think?
    - for concatenating, would need to check footprints and only take products with the same footprint, or subset them all to a common AOI? '''
    da = da_orig.copy()
    da = da.rename({'band_data': file_type}).squeeze()
    fname = os.path.basename(da_orig['band_data'].encoding['source'])

    sensor = fname[0:3]
    beam_mode = fname[4:6]
    acq_date_raw = fname[7:22]  # need to parse further
    acq_date = datetime.strptime(acq_date_raw, '%Y%m%dT%H%M%S')
    acq_time = fname[15:22]
    pol_type = fname[24:25]  # dual pol ...
    primary_pol = fname[25:26]  # VV ...
    orbit_type = fname[26:27]  # Precise (P), Restituted (R), or Original Predicted (O)
    terrain_correction_pixel_spacing = fname[27:32]  # Terrain Correction Pixel Spacing
    output = fname[35]  # Gamma-0 (g) or Sigma-0 (s) Output
    output_type = fname[36]  # Power (p) or Decibel (d) or Amplitude (a) Output
    masked = fname[37]  # Unmasked (u) or Water Masked (w)
    filtered = fname[38]  # Not Filtered (n) or Filtered (f)
    area = fname[39]  # Entire Area (e) or Clipped Area (c)
    product_id = fname[42:46]  # Product ID

    da.attrs = {'sensor': sensor,
                  'beam_mode': beam_mode,
                  'acquisition_date': acq_date,
                  'acquisition_time': acq_time,
                  'polarisation_type': pol_type,
                  'primary_polarisation': primary_pol,
                  'orbit_type': orbit_type,
                  'terrain_correction_pixel_spacing': terrain_correction_pixel_spacing,
                  'output_format': output,
                  'output_type': output_type,
                  'masked': masked,
                  'filtered': filtered,
                  'area': area,
                  'product_id': product_id
                  }
    utm_zone = da.spatial_ref.attrs['crs_wkt'][17:29]
    epsg_code = da.spatial_ref.attrs['crs_wkt'][589:594]

    da.attrs['utm_zone'] = utm_zone
    da.attrs['epsg_code'] = f'EPSG:{epsg_code}'

    date = da.attrs['acquisition_date']

    da = da.assign_coords({'acq_date': date})
    da = da.expand_dims('acq_date')
    da = da.drop_duplicates(dim=['x', 'y'])

    return da

def preprocess_vv(data):
    return preprocess(data, file_type='vv')

def preprocess_vh(data):
    return preprocess(data, file_type='vh')

def preprocess_ls(data):
    return preprocess(data, file_type='ls')

asf_vv = xr.open_mfdataset(paths = fpaths_vh, preprocess = preprocess_vv, chunks = 'auto', engine='rasterio', data_vars='minimal', coords='minimal', concat_dim='acq_date', combine='nested', parallel=True)
asf_vh = xr.open_mfdataset(paths = fpaths_vh, preprocess = preprocess_vh, chunks = 'auto', engine='rasterio', data_vars='minimal', coords='minimal', concat_dim='acq_date', combine='nested', parallel=True)
asf_ls = xr.open_mfdataset(paths = fpaths_vh, preprocess = preprocess_ls, chunks = 'auto', engine='rasterio', data_vars='minimal', coords='minimal', concat_dim='acq_date', combine='nested', parallel=True)

#TODO: COMBINE DATASET OR REMOVE UNNCECESARRY FILE TYPES

Create a temporal average for both pre and post EQ

In [ ]:
import numpy as np
def power_to_db(input_arr):
    return 10*np.log10(np.abs(input_arr))

date_bins = [start_time, event_time, end_time]
date_bin_labels = ["preevent", "postevent"]

asf_vv = asf_vv.groupby_bins("acq_date", date_bins, labels=date_bin_labels)
asf_vh = asf_vh.groupby_bins("acq_date", date_bins, labels=date_bin_labels)
asf_ls = asf_ls.groupby_bins("acq_date", date_bins, labels=date_bin_labels)

mean_vv = asf_vv.mean(dim='acq_date')
mean_vh = asf_vh.mean(dim='acq_date')
mean_ls = asf_vv.mean(dim='acq_date')


Perform a log difference of the two resulting images.

In [ ]:
log_preevent = np.log10(mean_vv['vv'][0])
log_postevent = np.log10(mean_vv['vv'][1])

log_diff = log_postevent - log_preevent

Discuss options for thresholding the difference image

Use the COP30 DEM to remove change detections in areas with slopes <5 degrees use gdaldem

In [ ]:
from asf_tools.dem import prepare_dem_vrt
from osgeo import ogr

dem_file = data_dir / 'DEM.tif'
prepare_dem_vrt(dem_file, ogr.CreateGeometryFromWkt(wkt))
process_dem_file = data_dir / 'DEM_processed.tif'
gdal.DEMProcessing(destName=str(process_dem_file), srcDS=str(dem_file), processing="slope", format="Gtiff", slopeFormat="degree")

dem = gdal.Open(str(process_dem_file))

 Plot a histogram for the difference image (might have to subset histogram source to area with equal amounts of landslides and non-landslides).

Create a figure to interactively view the data and set a threshold using matplotlib